# Model counting validation
Use annotated images from dataset to validate the counting test

In [ ]:
from analyze import analyze_validate
from scg_detection_tools.models import YOLOv8, YOLO_NAS
from scg_detection_tools.utils.file_handling import get_all_files_from_paths

MODEL_PATH = "/home/juliocesar/leaf-detection/checkpoints/yolov10/large/latest/last.pt"
CONFIDENCE = 50.0
model = YOLOv8(MODEL_PATH)

IMG_SOURCE = "/home/juliocesar/leaf-detection/datasets/gen-leaf-dataset-v7/valid/images"
ANN_SOURCE = "/home/juliocesar/leaf-detection/datasets/gen-leaf-dataset-v7/valid/labels"
imgs = get_all_files_from_paths(IMG_SOURCE)

analyze_validate(model, imgs, ANN_SOURCE, CONFIDENCE)

# Leaf counting and plot

In [ ]:
from analyze import analyze_count
from scg_detection_tools.utils.file_handling import get_all_files_from_paths, read_yaml
from scg_detection_tools.detect import Detector
from scg_detection_tools.models import YOLOv8, YOLO_NAS

import os
from pathlib import Path


cfg = read_yaml("analyze_config.yaml")

#model = YOLOv8(cfg["yolov8_model_path"])
model = YOLO_NAS(cfg["yolonas_arch"], cfg["yolonas_model_path"], cfg["data_classes"])
detector = Detector(model, cfg["detect_parameters"], specific_det_params={"use_slice": False})

IMG_DIR = os.path.abspath("/home/juliocesar/leaf-detection/imgs/scgtestes_c1at4")

def sort_stem(item):
    s = Path(item).stem
    return int(s) if s.isnumeric() else s

imgs = get_all_files_from_paths(IMG_DIR)
imgs = sorted(imgs, key=sort_stem)

res = analyze_count(detector, 
                    imgs, 
                    save_detections=True,
                    plot=False,
                    per_image_return=True)

print(res)

# Color Analysis

In [ ]:
from analyze import analyze_color_histogram
from scg_detection_tools.models import YOLO_NAS, YOLOv8
from scg_detection_tools.detect import Detector
from scg_detection_tools.utils.file_handling import read_yaml
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import cv2

cfg = read_yaml("analyze_config.yaml")
model = YOLOv8(yolov8_ckpt_path=cfg["yolov8_model_path"])
#model = YOLO_NAS(model_arch=cfg["yolonas_arch"], 
#                checkpoint_path=cfg["yolonas_model_path"], 
#                classes=cfg["data_classes"])
det_params = cfg["detect_parameters"]
det_params["embed_slice_callback"] = None
det = Detector(detection_model=model, detection_params=det_params)


In [ ]:
from scg_detection_tools.utils.file_handling import get_all_files_from_paths

IMG_DIR = "/home/juliocesar/leaf-detection/imgs/light_group/images"
LBL_DIR = "/home/juliocesar/leaf-detection/imgs/light_group/labels"

#IMG_DIR = "/home/julio/Dev/SCG_IFSC/save/hemacias/images"

imgs = get_all_files_from_paths(IMG_DIR)

img_hists = analyze_color_histogram(model=model, 
                                    detector=det, 
                                    imgs=imgs, 
                                    raw=False, 
                                    on_detection_boxes=False,
                                    seg_annotations=LBL_DIR, 
                                    cspaces=["RGB", "HSV", "GRAY"], 
                                    show=True, save_plots=False)


In [ ]:
# Plot single channel images

import os

save = False
for img in imgs:
    fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12,8))
    
    rgb = cv2.imread(img)
    rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(rgb.copy(), cv2.COLOR_RGB2HSV)
    LABELS = ["RGB", "HSV"]
    for ci, cp in enumerate([rgb, hsv]):
        for i in range(3):
            ch = cp.copy()
            for j in range(3):
                if i != j:
                    ch[:,:,j] = 0
            axs[ci][i].axis("off")
            axs[ci][i].imshow(ch)
            axs[ci][i].set_title(LABELS[ci][i])
    if save:
        fig.savefig(f"exp_analysis/singlechannel_{os.path.basename(img)}")
    plt.show()

In [ ]:
# Plot RGB, HSV and Gray mean and std. Also plot full scatter matrix

cspaceAnalysis = {"RGB": [], "HSV": [], "GRAY": [], "ALL": []}
nclass = {0: "light", 1: "medium", 2: "dark", 3: "dead"}
intensities = np.arange(256)
for img in img_hists:
    img_hist = img_hists[img]
    for cspace in ["RGB", "HSV", "GRAY"]:
        for mask_hist in img_hist[cspace]["masks"]:
            hist = mask_hist["hist"]
            mask_class = nclass[mask_hist["class"]]

            ch_stats = []
            for ch_hist in hist:
                ch_mean = np.sum(intensities * ch_hist) / np.sum(ch_hist)
                ch_std = np.sqrt(np.sum((intensities - ch_mean) ** 2 * ch_hist) / np.sum(ch_hist))
                ch_stats.append([ch_mean, ch_std])
            
            ch_stats = np.array(ch_stats).T.ravel().tolist()
            ch_stats.append(mask_class)
            cspaceAnalysis[cspace].append(ch_stats)


num_masks = len(cspaceAnalysis["RGB"])
all_mean = []
all_std = []
all_class = [data[-1] for data in cspaceAnalysis["RGB"]]

for maskidx in range(num_masks):
    all_mean.append([])
    all_std.append([])
    for cspace in ["RGB", "HSV", "GRAY"]:
        data = cspaceAnalysis[cspace][maskidx]
        if cspace == "GRAY":
            ch_mean = data[0]
            ch_std = data[1]
            all_mean[-1].append(ch_mean)
            all_std[-1].append(ch_std)
        else:
            ch_mean = data[:3]
            ch_std = data[3:-1]
            all_mean[-1].extend(ch_mean)
            all_std[-1].extend(ch_std)
for i in range(num_masks):
    cspaceAnalysis["ALL"].append([])
    
    cspaceAnalysis["ALL"][i].extend(all_mean[i])
    cspaceAnalysis["ALL"][i].extend(all_std[i])
    cspaceAnalysis["ALL"][i].append(all_class[i])

rgbdf = pd.DataFrame(cspaceAnalysis["RGB"], columns=[
    "R Mean", "G Mean", "B Mean", "R std", "G std", "B std", "Class"
])
hsvdf = pd.DataFrame(cspaceAnalysis["HSV"], columns=[
    "H Mean", "S Mean", "V Mean", "H std", "S std", "V std", "Class"
])
graydf = pd.DataFrame(cspaceAnalysis["GRAY"], columns=[
    "Gray mean", "Gray std", "Class"
])

alldf = pd.DataFrame(cspaceAnalysis["ALL"], columns=[
    "R Mean", "G Mean", "B Mean", "H Mean", "S Mean", "V Mean", "Gray mean", "R std", "G std", "B std", "H std", "S std", "V std", "Gray std", "Class"
])

class_c_map = {
    "light": "blue", 
    "medium": "green", 
    "dark": "red", 
    "dead": "black"
}

class_color = lambda df: df["Class"].map(class_c_map)

scatter_matrix(rgbdf.iloc[:,:-1], figsize=(12,12), diagonal="kde", color=class_color(rgbdf))
scatter_matrix(hsvdf.iloc[:,:-1], figsize=(12,12), diagonal="kde", color=class_color(hsvdf))
scatter_matrix(graydf.iloc[:,:-1], figsize=(12,12), diagonal="kde", color=class_color(graydf))
scatter_matrix(alldf.iloc[:,:-1], figsize=(20,20), diagonal="kde", color=class_color(alldf))

print(rgbdf.corr(numeric_only=True))
print(hsvdf.corr(numeric_only=True))
print(graydf.corr(numeric_only=True))

# Leaf color identification pipeline (light, medium, dark, dead)

In [ ]:
#########################################################
### ANALYSIS CLASSIFICATION TEST
#########################################################

from scg_detection_tools.models import YOLO_NAS, YOLOv8
from scg_detection_tools.detect import Detector
from scg_detection_tools.utils.file_handling import read_yaml, get_all_files_from_paths

cfg = read_yaml("analyze_config.yaml")
#model = YOLO_NAS(model_arch=cfg["yolonas_arch"], 
#                checkpoint_path=cfg["yolonas_model_path"], 
#                classes=cfg["data_classes"])
model = YOLOv8(yolov8_ckpt_path=cfg["yolov8_model_path"])
det_params = cfg["detect_parameters"]
det_params["embed_slice_callback"] = None
det = Detector(detection_model=model, detection_params=det_params)


from analyze import analyze_classify

imgs = [
    "/home/juliocesar/leaf-detection/scg-leaf-count/imgs/test/5_640x640_aglom.png",
    "/home/juliocesar/leaf-detection/scg-leaf-count/imgs/test/0_640x640.png",
    "/home/juliocesar/leaf-detection/scg-leaf-count/imgs/test/6_640x640.png",
]

hem_imgs = get_all_files_from_paths("/home/juliocesar/leaf-detection/imgs/hemacias/scaled")
hem_labels = ["purple", "white"]
hem_colors = {
    "purple": "yellow",
    "white": "blue",
}

leaf_labels = ["dark", "dead", "light", "medium"]
leaf_colors = {
    "light": "deepskyblue",
    "medium": "yellow",
    "dark": "red",
    "dead": "black",
}

analyze_classify(detector=det,
                 imgs=imgs,
                 cls_labels=leaf_labels,
                 cls_colors=leaf_colors,
                 method="knn",
                 sam2_ckpt_path=cfg["sam2_ckpt_path"],
                 sam2_cfg=cfg["sam2_cfg"])


# Pixel density

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
from analyze import analyze_pixel_density
from scg_detection_tools.utils.file_handling import get_all_files_from_paths
from pathlib import Path
from scg_detection_tools.models import YOLO_NAS, YOLOv8
from scg_detection_tools.detect import Detector
from scg_detection_tools.utils.file_handling import read_yaml

cfg = read_yaml("analyze_config.yaml")
#model = YOLOv8(yolov8_ckpt_path=cfg["yolov8_model_path"])
model = YOLO_NAS(model_arch=cfg["yolonas_arch"], 
                checkpoint_path=cfg["yolonas_model_path"], 
                classes=cfg["data_classes"])
det_params = cfg["detect_parameters"]
det_params["embed_slice_callback"] = None
det = Detector(detection_model=model, detection_params=det_params)

IMG_DIR = "/home/juliocesar/leaf-detection/scg-leaf-count/imgs/analysis"
#LBL_DIR = "/home/juliocesar/leaf-detection/save/first_analysis_detections/out_cache/"

imgs = get_all_files_from_paths(IMG_DIR)

def sort_stem(item):
    s = Path(item).stem
    try:
        val = int(s)
        return val
    except:
        return s

imgs = sorted(imgs, key=sort_stem)
print(imgs)

densities = analyze_pixel_density(model=model, 
                                  detector=det, 
                                  imgs=imgs, 
                                  sam2_ckpt_path=cfg["sam2_ckpt_path"],
                                  sam2_cfg=cfg["sam2_cfg"],
                                  boxes=True,
                                  segments=False,
                                  slice_detection=True,
                                  on_slice=True,
                                  seg_annotations=None,
                                  cached_detections=None,
                                  show=True,
                                  save=False)
print(densities)